# Aqua 0.7 Operator Refactor
_5-Dec-19, donny@, ..._

## Purpose
To improve the transparency and ease of understanding of Qiskit’s operator logic and algorithmic usage. Specifically, to reconcile with the Terra operator hierarchy and make the ExpectationValue and Evolution algorithms more visible, explicit, and extensible.

## Present State (Aqua 6.1)
The Operator is the workhorse of Aqua, containing both the expectation value and evolution logic used by most of the algorithms in Aqua. However this logic is spread across the operator, very branchy for different modes of execution, and not built for user modification to the expectation value or evolution logic.

The ExpectationValue is not contained within a single function or module, but rather split into several functions without a clear interface or flow for user usage. This is due to structural constraints in Aqua which are no longer present, where the algorithms requiring the expectation value held the backend and could run circuits, but the operator could not. Presently, the code to take an expectation value in VQE is too long to paste here, but we encourage the user to read lines 361-395 of Aqua 6.1 VQE’s ExpectationValue calculation to try to understand where and how the expectation is computed. We’ve been asked by numerous Aqua users to explain how this code works, and most do not attempt to use it on their own.

...

### Vanilla Expectation Example

The following is the shortest possible way to write an expectation value in Aqua. Note that it fundamentally requires the user to understand a certain execution flow, the correct functions to use to do this, and how those functions work with their execute mode. This takes a few hours to understand at least, often days. Further, there are no hints that tomography is being performed here, or matrix multiplication if the system chooses to do that instead.

In [7]:
from qiskit.aqua.operators import WeightedPauliOperator
from qiskit.aqua.components.variational_forms import RY
from qiskit.quantum_info import Pauli
from qiskit import BasicAer, execute, QuantumCircuit
from qiskit.circuit import Parameter
qasm_sim = BasicAer.get_backend('qasm_simulator')

In [2]:
op = WeightedPauliOperator([
    [.5j, Pauli.from_label('IX')],
    [.2, Pauli.from_label('ZY')],
])
circuit = QuantumCircuit(2)
circuit.h([0,1])

evaluation_circuits = op.construct_evaluation_circuit(wave_function=circuit, statevector_mode=False)
result = execute(evaluation_circuits, qasm_sim).result()
expect, std = op.evaluate_with_result(result=result, statevector_mode=False)
expect

(-0.004687500000000001+0.5j)

### Vanilla Evolution Example

...

In [5]:
# θ = Parameter('θ')
instr = op.evolve_instruction(evo_time=.5)
circuit.append(instr, [0,1])
circuit.decompose().draw(fold=2000)

### Missing Functionality: Tasks Not Possible Today

...

### Overlaps with Terra and Ignis, and Code Organization Issues
...

## Core Requirements / Interface

1. Reconciliation with Terra Operators
1. ExpectationValue and Evolution algorithm hierarchies
1. Usage of Operator Abstractions within the Aqua Algorithms
1. Aer ExpectationValue and Evolution
1. Reconciliation with Ignis Tomography
1. Smooth Borders with Application Stack Primitives

### Reconciling the Operator logic in Aqua with that of Terra. 

Terra contains a suite of Matrix-represented operators within the quantum_info module, including rich logic for combining and manipulating various operator types. The Aqua operator does not need to replicate this logic, and is the not the only place in Qiskit which requires it for non-matrix representation (e.g. WeightedPauli) Operators. The mathematical logic in Aqua's operators should be moved into Terra, and Aqua should use these as its core operator objects.

...

In [ ]:
from qiskit.quantum_info import WeightedPauliOperator

### Breaking out the ExpectationValue and Evolution to be explicit modules which can be interchanged and extended.

Users want to easily find and understand ExpectationValue and Evolution logic, and understand the algorithmic variations available to them.

...

In [ ]:
from qiskit.aqua.algorithms import ExpectationValue, Evolution

avg = ExpectationValue(operator, backend).run(circuit)
instr = Evolution(operator, backend).instruction(param)

### Terse and transparent Operator usage within the Aqua algorithms, characteristic of end-user usage of the same algorithms.

...

In [ ]:
class VQE(QuantumAlgorithm):
    def __init__(self, operator, var_form, optimizer,
                 initial_point=None, backend=backend, callback=None, ...):
        ...
        self._expectation_value = ExpectationValue(self._operator, self._backend)
    
    def _energy_evaluation(self, params):
        circuits = self._var_form.construct_circuit(params)
        energy, stdev = self._expectation_value.run(circuits)
        return energy

### Explicit modules for Aer's ExpectationValue and Evolution which can be swapped for Aqua's, and a framework for remote backends to provide RemoteAlgorithm modules without depending on Aqua. 

...

In [ ]:
from qiskit.providers.aer import AerExpectationValue

my_vqe = VQE()

### Reconciling the implicit tomography in Aqua's expectation value with Ignis's tomography functionality to avoid code duplication and allow generic access to the functionality.

...

### Clear and smooth borders with the Operator-related objects in Aqua, such as the FermionicOperator and Hamiltonian in Chemistry, and the proposed LadderOperator and OptimizationProblem.

...

## Technical Design

* Changes to Terra Operator Hierarchy
* Aqua Algorithm Hierarchy
    * Groups and Symmetries Options
* Changes to Ignis
* Qiskit Remote Algorithms
* Aer ExpectationValue and Evolution
* IBMQ or other backend RemoteAlgorithms

### Changes to Terra Operator Hierarchy

There are two options for the migration of the Operator mathematical logic into Terra's quantum_info:

#### Option A: Move WeightedPauliOperator into quantum_info and refactor

...

#### Option B: Add Linear Combination functionality to Operator to allow for WeightedPauli, WeightedClifford, and Combinations

* e.g. Pauli a + Pauli b = LinComb(a, b)
* Circuit Operator?

...

...

## Timeline and Gameplan

...

## Future Extension Opportunities